In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from split_llm.glm6b.wrapped_layer import Attention_GLM_Wrapped, copy_attention

/home/zf/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

NameError: name 'transformer_ls' is not defined

In [ ]:
transformer_layer = glm.condgen.transformer.layers[0].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, 0)

copy_attention(transformer_layer, attn_wrapped)

In [ ]:
attn_wrapped.cuda()

In [ ]:
import torch

In [ ]:
x = torch.normal(0, 1, [10, 1, 4096]).cuda()

In [ ]:
h_qkv = x @ attn_wrapped.qkv_weight.T

In [ ]:
try:
    import sys
    del sys.modules['split_llm.glm6b.utils']
    del sys.modules['split_llm.protocols.base_protocols']
    del sys.modules['split_llm.protocols.element_wise']
    del sys.modules['split_llm.glm6b.secure_inference']
    print("delete complete!")
except Exception as e:
    print(e)
from split_llm.glm6b.secure_inference import GLMAttentionProtocol

In [ ]:
transformer_layer = glm.condgen.transformer.layers[0].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, 0)

copy_attention(transformer_layer, attn_wrapped)

In [ ]:
from split_llm.common.communication import Communication, Node, SimulatedCommunication
communication = SimulatedCommunication(["n0", "n1", "n2"])
communication.new_stage("Test")

n0 = Node(communication, "n0")
n1 = Node(communication, "n1")
n2 = Node(communication, "n2")

In [ ]:
n0.space.attentions = [attn_wrapped]

In [ ]:
protocol = GLMAttentionProtocol(
    n0, n1, n2, 0, 10, device="cuda"
)
protocol.prepare()
protocol.offline_execute(10, 10)

In [ ]:
print(sys.modules.keys())